# OpenAI Whisper Fine-tuning for Korean ASR with HuggingFace Transformers

본격적인 작업에 앞서, Colab을 사용하여 본 작업을 수행한다면 데이터의 전처리 작업까지는 CPU를 활용하는 것을 권장한다. 데이터 전처리 후 Huggingface에 전처리된 데이터셋을 업로드하고, 그 후에 런타임 유형을 GPU로 변환하여 전처리된 데이터셋을 로드, 모델의 학습을 수행한다면 Colab 컴퓨팅 자원의 소모를 방지할 수 있을 것이다.

### Prepare Environment

In [70]:
!uv pip install datasets
!uv pip install git+https://github.com/huggingface/transformers
!uv pip install evaluate
!uv pip install jiwer
!uv pip install accelerate -U
!uv pip install transformers==4.50.1
!uv pip install wandb
!uv pip install ipywidgets
!uv pip install pymysql
!uv pip install sqlalchemy
!uv pip install librosa
!uv pip install tensorboard
!uv pip install tensorboardX

Resolved 33 packages in 141ms
Uninstalled 1 package in 16ms
Installed 1 package in 14ms
 - fsspec==2025.5.1
 + fsspec==2025.3.0
Resolved 18 packages in 1.03s
Uninstalled 1 package in 454ms
Installed 1 package in 586ms
 - transformers==4.50.1
 + transformers==4.54.0.dev0 (from git+https://github.com/huggingface/transformers@ebfbcd42da327b4a9f2d73c93a962be0a581faaa)
Audited 1 package in 10ms
Audited 1 package in 5ms
Resolved 23 packages in 66ms
Prepared 1 package in 0.42ms
Uninstalled 1 package in 12ms
Installed 1 package in 13ms
 - fsspec==2025.3.0
 + fsspec==2025.5.1
Resolved 18 packages in 46ms
Uninstalled 1 package in 596ms
Installed 1 package in 327ms
 - transformers==4.54.0.dev0 (from git+https://github.com/huggingface/transformers@ebfbcd42da327b4a9f2d73c93a962be0a581faaa)
 + transformers==4.50.1
Audited 1 package in 8ms
Audited 1 package in 10ms
Audited 1 package in 6ms
Audited 1 package in 8ms
Audited 1 package in 8ms
Resolved 12 packages in 228ms
Prepared 6 packages in 1.66s
Ins

### Prepare Feature Extractor and Tokenizer

음성 인식 Pipeline은 다음과 같이 세 가지 구성으로 나눌 수 있다.
1. 오디오 raw data에 대하여 pre-process를 진행할 feature extractor
2. sequence-to-sequence 매핑을 수행할 model
3. model의 output을 text 포맷으로 후처리할 tokenizer

HuggingFace의 Transformers는 Whisper의 모델과 함께 이와 연관된 WhisperFeatureExtractor와 WhisperTokenizer를 함께 제공하고 있다.

#### Load WhisperFeatureExtractor

발화는 시간에 따라 변화하는 1차원 배열로 표현된다. 각 시계열별로 입력된 배열의 값은 그 순간의 신호가 갖는 진폭이다. 그 진폭의 정보만으로, 우리는 음성 데이터의 주파수 스펙트럼을 재구성하고 모든 음향적 특징을 복원할 수 있다.

발화는 연속적이므로, 진폭의 값은 무한하다. 그리고 이는 유한한 배열을 기대하는 컴퓨터 장치에 문제를 야기한다. 따라서, 우리는 고정된 시간 간격에 따라 신호에서 값을 샘플링하여 우리의 음성 신호를 분할한다. 오디오를 샘플링하는 간격은 sampling rate로 알려져 있으며, 이는 일반적으로 '샘플/초' 또는 헤르츠(Hz) 단위로 측정된다. 높은 sample rate로 샘플링하는 것은 연속적인 발화 신호에 대하여 더 나은 접근을 가능케 하지만, 초당 더 많은 값을 저장하도록 요구하기도 한다.

우리가 가진 오디오 데이터의 sampling rate를 우리가 사용할 모델이 기대하는 sampling rate와 일치시키는 것은 매우 중요한데, 서로 다른 sampling rate의 오디오 신호는 매우 다른 분포를 보이기 때문이다. 오디오 신호는 적절한 sampling rate로 처리되어야만 한다. 그렇지 않으면 예상치 못한 결과를 야기할 수 있다. 예를 들어, 16kHz로 샘플링된 오디오 샘플을 8kHz의 sampling rate로 들으면 해당 오디오의 속도가 절반으로 들릴 것이다. 동일한 방식으로, 잘못된 sampling rate의 오디오 데이터를 ASR 모델에 넘길 경우 모델이 불안정해질 수 있다. Whisper Feature Extractor는 16kHz의 오디오 데이터를 요구하며, 우리는 우리가 가진 오디오 데이터를 이에 맞추어야 한다.

Whisper Feature Extractor는 두 가지 동작을 수행한다. 먼저 오디오 샘플에 대하여 padding 및 trucating을 진행하여 모든 샘플들의 길이를 30초로 맞춘다. 30초보다 짧은 샘플은 0으로 채워 30초로 맞추고(음성 신호에서 0은 신호 없음 혹은 침묵을 의미한다), 30초보다 긴 샘플들은 30초로 잘라낸다. 이처럼 인풋 단계에서 각 배치의 모든 요소들이 최대 길이로 채워지거나 잘라지기 때문에, 우리는 Whisper 모델에 데이터를 전달할 때 Attention mask를 필요로 하지 않는다. Whisper 모델은 이 부분에서 독특한데, 대부분의 오디오 모델은 sequence가 패딩된 위치와 self-attention 메커니즘이 어느 부분을 무시해야 하는지에 대한 상세 정보를 제공하는 attention mask를 필요로 한다. 그러나 Whisper는 이러한 attention mask 없이 발화 신호로부터 무시해야 할 인풋을 직접 추론하도록 학습되었다.

Whisper Feature Extractor의 두 번째 동작은 패딩된 오디오 배열을 log-Mel 스펙트럼으로 변환하는 것이다. 이 스펙트럼은 푸리에 변환과 같이 주파수 신호를 시각적으로 나타낸 것이다.Whisper 모델의 인풋으로 주어져야 하는 것이 바로 이 log-Mel 스펙트럼이다.

Transformer의 Whisper Feature Extractor는 패딩과 스펙트로그램 변환을 단 한 줄의 코드로 수행할 수 있다.

In [50]:
from transformers import WhisperFeatureExtractor

whisper_model_name = "openai/whisper-tiny"

# 파인튜닝을 진행하고자 하는 모델의 feature extractor를 로드
feature_extractor = WhisperFeatureExtractor.from_pretrained(whisper_model_name)

#### Load WhisperTokenizer

Whisper 모델의 아웃풋은 vocabulary item들의 단어 중 예측된 텍스트를 나타내는 index값이다. Tokenizer는 이러한 텍스트 토큰 시퀀스와 실제 텍스트 문자열을 매핑해준다.

In [51]:
from transformers import WhisperTokenizer
# 파인튜닝을 진행하고자 하는 모델의 tokenizer를 로드
tokenizer = WhisperTokenizer.from_pretrained(whisper_model_name, language="Korean", task="transcribe")
tokenizer

WhisperTokenizer(name_or_path='openai/whisper-tiny', vocab_size=50258, model_max_length=1024, is_fast=False, padding_side='right', truncation_side='right', special_tokens={'bos_token': '<|endoftext|>', 'eos_token': '<|endoftext|>', 'unk_token': '<|endoftext|>', 'pad_token': '<|endoftext|>', 'additional_special_tokens': ['<|endoftext|>', '<|startoftranscript|>', '<|en|>', '<|zh|>', '<|de|>', '<|es|>', '<|ru|>', '<|ko|>', '<|fr|>', '<|ja|>', '<|pt|>', '<|tr|>', '<|pl|>', '<|ca|>', '<|nl|>', '<|ar|>', '<|sv|>', '<|it|>', '<|id|>', '<|hi|>', '<|fi|>', '<|vi|>', '<|he|>', '<|uk|>', '<|el|>', '<|ms|>', '<|cs|>', '<|ro|>', '<|da|>', '<|hu|>', '<|ta|>', '<|no|>', '<|th|>', '<|ur|>', '<|hr|>', '<|bg|>', '<|lt|>', '<|la|>', '<|mi|>', '<|ml|>', '<|cy|>', '<|sk|>', '<|te|>', '<|fa|>', '<|lv|>', '<|bn|>', '<|sr|>', '<|az|>', '<|sl|>', '<|kn|>', '<|et|>', '<|mk|>', '<|br|>', '<|eu|>', '<|is|>', '<|hy|>', '<|ne|>', '<|mn|>', '<|bs|>', '<|kk|>', '<|sq|>', '<|sw|>', '<|gl|>', '<|mr|>', '<|pa|>', '<|si|

WhisperTokenizer는 인코딩 과정에서 'Special token'들을 부여한다. 여기에는 문장의 시작과 끝을 나타내는 토큰(전사의 시작과 끝을 나타내는 토큰을 포함한다), 언어를 나타내는 토큰, 태스크(전사, 번역 등)를 나타내는 토큰 등이 있다.

In [52]:
input_str = "저는 서울중앙지검 지능범죄수사팀 최인호 검사입니다."
labels = tokenizer(input_str).input_ids
print(f"Input IDs: {labels}")
decoded_with_special = tokenizer.decode(labels, skip_special_tokens=False)
decoded_str = tokenizer.decode(labels, skip_special_tokens=True)

print(f"Input:                 {input_str}")
print(f"Decoded w/ special:    {decoded_with_special}")
print(f"Decoded w/out special: {decoded_str}")
print(f"Are equal:             {input_str == decoded_str}")

Input IDs: [50258, 50264, 50359, 50363, 11738, 1098, 31039, 13907, 48808, 1831, 1368, 222, 4704, 16102, 15810, 242, 3354, 226, 8449, 5727, 169, 3638, 14571, 4215, 14705, 20282, 5727, 7416, 13, 50257]
Input:                 저는 서울중앙지검 지능범죄수사팀 최인호 검사입니다.
Decoded w/ special:    <|startoftranscript|><|ko|><|transcribe|><|notimestamps|>저는 서울중앙지검 지능범죄수사팀 최인호 검사입니다.<|endoftext|>
Decoded w/out special: 저는 서울중앙지검 지능범죄수사팀 최인호 검사입니다.
Are equal:             True


#### Combine To Create a WhisperProcessor

Feature Extactor와 Tokenizer를 간편히 사용하기 위해, 두 모듈을 WhisperProcessor 클래스 하나로 묶을 수 있다. Processor는 Feature Extactor와 Tokenizer를 상속하며, 오디오 인풋의 입력과 모델의 예측에 사용할 수 있다. 이로써 우리는 훈련 과정에서 Processor와 Model이라는 두 개의 객체만 추적하면 된다.

In [53]:
from transformers import WhisperProcessor

processor = WhisperProcessor.from_pretrained(whisper_model_name, language="Korean", task="transcribe")

### Prepare Data

보유한 음성 데이터와 이에 매치되는 텍스트 전사 데이터를 모델에 입력할 수 있는 형태로 변환해야 한다. 앞서 언급했듯 WhisperFeatureExtractor는 16kHz의 오디오 데이터를 입력으로 받으므로 이에 맞추어 오디오 데이터를 리샘플링해야 한다.

SBS DB에서 필요한 데이터를 읽는 과정이 필요하다. \
이 데이터에 대하여 수행해야 할 전처리를 단계별로 구분하면 다음과 같다.

1. 'idx, 오디오 파일 경로, transcription' 순으로 하나의 인스턴스가 만들어지도록 DB 내 데이터를 취합하여 DataFrame으로 만든다.
2. 오디오 파일 경로를 이용해 sampling rate를 확인하고, 필요할 경우 16kHz로 리샘플링한다.
3. 상기의 DataFrame을 input(오디오 경로)과 label(transcription)로 분류하여 x, y 데이터로 분할한다.

#### 파일 경로 취합

In [54]:
import sys
import os
import pymysql
from sqlalchemy import create_engine

server_address = "10.10.210.206" # e.g., "192.168.1.100" or "server.local"
def fix_wav_filepath_win(wav_filepath):
    if '/cul_m01/' in wav_filepath and 'Y:' not in wav_filepath:
        new_wav_filepath = r"\\" + server_address + wav_filepath
    else:
        new_wav_filepath = wav_filepath
    return new_wav_filepath

class ProdDB():
    def __init__(self):
        # set database connection
        self.default_db = {
            "default_db_mode": 'production',
            "default_db_name": 'sbs_asr_datas',
            "default_db_host": '10.10.110.200',
            "default_db_user": 'root',
            "default_db_passwd": 'sbsRnd1!'
        }
        self.conn = pymysql.connect(host=self.default_db['default_db_host'], 
                                  user=self.default_db['default_db_user'], 
                                  password=self.default_db['default_db_passwd'], 
                                  db=self.default_db['default_db_name'], 
                                  charset='utf8')
                                  
        self.cursor = self.conn.cursor()

        host=self.default_db['default_db_host']
        user=self.default_db['default_db_user']
        password=self.default_db['default_db_passwd']
        schema=self.default_db['default_db_name']
        port=3306
        self.cnx = create_engine(f'mysql+pymysql://{user}:{password}@{host}:{port}/{schema}', echo=False)

        print('TestDB', self.default_db['default_db_host'])

    def execute_many(self, sqlcmd, val=None):
        self.cursor.executemany(sqlcmd, val)
        self.conn.commit()
        return True
    
    def execute_one(self, sqlcmd, val=None):
        self.cursor.execute(sqlcmd, val)
        self.conn.commit()
        return True
    
    def get_count(self, table_name):
        sqlcmd = f'SELECT COUNT(*) FROM sbs_asr_datas.{table_name}'
        ret = self.cursor.execute(sqlcmd)
        row = self.cursor.fetchone()
        count = row[0]
        return count

    def select_data(self, sqlcmd):
        self.cursor.execute(sqlcmd)
        rows = self.cursor.fetchall()
        return rows
    
    def get_colnames(self):
        cols = [i[0] for i in self.cursor.description]
        return cols
    
    def finish_db(self):
        self.conn.close()
        self.cursor.close()
        self.cnx.dispose()
        return True

db = ProdDB()

sqlcmd = f"SELECT raw_data, transcript FROM sbs_asr_datas.training_datas LIMIT 10000"
datas = db.select_data(sqlcmd)
print(datas[:10])

raw_data_list = [fix_wav_filepath_win(data[0]) for data in datas]
transcript_list = [data[1] for data in datas]


TestDB 10.10.110.200
(('/cul_m01/프리뷰노트/SBS_폐쇄자막_데이터/audio_seg/PRGTM_202106270005_00001.wav', '우와, 이거 뭐 있는 것 같은데?'), ('/cul_m01/프리뷰노트/SBS_폐쇄자막_데이터/audio_seg/PRGTM_202106270005_00002.wav', '뭐지?'), ('/cul_m01/프리뷰노트/SBS_폐쇄자막_데이터/audio_seg/PRGTM_202106270005_00003.wav', '딱 봐도 캔이지.'), ('/cul_m01/프리뷰노트/SBS_폐쇄자막_데이터/audio_seg/PRGTM_202106270005_00004.wav', '우와, 진짜 오래됐네.'), ('/cul_m01/프리뷰노트/SBS_폐쇄자막_데이터/audio_seg/PRGTM_202106270005_00005.wav', '근데 되게 오래된 것 같아 그렇지?'), ('/cul_m01/프리뷰노트/SBS_폐쇄자막_데이터/audio_seg/PRGTM_202106270005_00006.wav', '옛날 거 아니야 옛날 거?'), ('/cul_m01/프리뷰노트/SBS_폐쇄자막_데이터/audio_seg/PRGTM_202106270005_00007.wav', '콜라인가?'), ('/cul_m01/프리뷰노트/SBS_폐쇄자막_데이터/audio_seg/PRGTM_202106270005_00008.wav', '야, 이거 봐. 이거 옛날 방식이잖아.'), ('/cul_m01/프리뷰노트/SBS_폐쇄자막_데이터/audio_seg/PRGTM_202106270005_00009.wav', '근데 이게 이렇게 안 썩는다 야.'), ('/cul_m01/프리뷰노트/SBS_폐쇄자막_데이터/audio_seg/PRGTM_202106270005_00010.wav', '저희가 쓰레기 줍는 산행하고 있는데 지금 힘드시겠지만 쓰레기봉투 좀 나눠드리면 한 5개~10개만이라도. 아, 감사합니다.'))


In [55]:
print(f"file_list : {raw_data_list[:10]}")
print(len(raw_data_list))

print(f"transcript_list : {transcript_list[:10]}")
print(len(transcript_list))

file_list : ['\\\\10.10.210.206/cul_m01/프리뷰노트/SBS_폐쇄자막_데이터/audio_seg/PRGTM_202106270005_00001.wav', '\\\\10.10.210.206/cul_m01/프리뷰노트/SBS_폐쇄자막_데이터/audio_seg/PRGTM_202106270005_00002.wav', '\\\\10.10.210.206/cul_m01/프리뷰노트/SBS_폐쇄자막_데이터/audio_seg/PRGTM_202106270005_00003.wav', '\\\\10.10.210.206/cul_m01/프리뷰노트/SBS_폐쇄자막_데이터/audio_seg/PRGTM_202106270005_00004.wav', '\\\\10.10.210.206/cul_m01/프리뷰노트/SBS_폐쇄자막_데이터/audio_seg/PRGTM_202106270005_00005.wav', '\\\\10.10.210.206/cul_m01/프리뷰노트/SBS_폐쇄자막_데이터/audio_seg/PRGTM_202106270005_00006.wav', '\\\\10.10.210.206/cul_m01/프리뷰노트/SBS_폐쇄자막_데이터/audio_seg/PRGTM_202106270005_00007.wav', '\\\\10.10.210.206/cul_m01/프리뷰노트/SBS_폐쇄자막_데이터/audio_seg/PRGTM_202106270005_00008.wav', '\\\\10.10.210.206/cul_m01/프리뷰노트/SBS_폐쇄자막_데이터/audio_seg/PRGTM_202106270005_00009.wav', '\\\\10.10.210.206/cul_m01/프리뷰노트/SBS_폐쇄자막_데이터/audio_seg/PRGTM_202106270005_00010.wav']
10000
transcript_list : ['우와, 이거 뭐 있는 것 같은데?', '뭐지?', '딱 봐도 캔이지.', '우와, 진짜 오래됐네.', '근데 되게 오래된 것 같아 그렇지?', '옛날 거 아니야 옛

스크립트를 데이터프레임에 추가한다.

In [8]:
import pandas as pd

df = pd.DataFrame(data=transcript_list, columns = ["transcript"])

In [9]:
# 텍스트 데이터로 만든 데이터프레임에 음성 파일 경로 컬럼을 추가
df["raw_data"] = raw_data_list

In [10]:
df

,transcript,raw_data
0,"우와, 이거 뭐 있는 것 같은데?",\\10.10.210.206/cul_m01/프리뷰노트/SBS_폐쇄자막_데이터/aud...
1,뭐지?,\\10.10.210.206/cul_m01/프리뷰노트/SBS_폐쇄자막_데이터/aud...
2,딱 봐도 캔이지.,\\10.10.210.206/cul_m01/프리뷰노트/SBS_폐쇄자막_데이터/aud...
3,"우와, 진짜 오래됐네.",\\10.10.210.206/cul_m01/프리뷰노트/SBS_폐쇄자막_데이터/aud...
4,근데 되게 오래된 것 같아 그렇지?,\\10.10.210.206/cul_m01/프리뷰노트/SBS_폐쇄자막_데이터/aud...
...,...,...
9995,붙는데요. 붙었을 때는 또 얼굴 하나. 역이용하면 좋겠어요.,\\10.10.210.206/cul_m01/프리뷰노트/SBS_폐쇄자막_데이터/aud...
9996,지금 위험했어요. 짧게라도 차야 돼요.,\\10.10.210.206/cul_m01/프리뷰노트/SBS_폐쇄자막_데이터/aud...
9997,뒤. 안 넘어졌어요.,\\10.10.210.206/cul_m01/프리뷰노트/SBS_폐쇄자막_데이터/aud...
9998,넘어지면 안 돼요. 장준.,\\10.10.210.206/cul_m01/프리뷰노트/SBS_폐쇄자막_데이터/aud...


In [11]:
# Null data 유무 확인
df.isnull().values.sum()

np.int64(0)

#### Transformer Dataset 만들기

취합한 데이터프레임을 Transformers의 DatasetDict로 만들어 두면 해당 데이터를 언제 어디서든 자유롭게 활용할 수 있다. 물론 컴퓨팅 환경이 적합하다면 오프라인이 훨씬 빠르겠지만, 그렇지 않다면 고려해볼 수 있는 사항이다.

In [12]:
from datasets import Dataset, DatasetDict
from datasets import Audio

In [13]:
# 오디오 파일 경로를 dict의 "audio" 키의 value로 넣고 이를 데이터셋으로 변환
# 이때, Whisper가 요구하는 사양대로 Sampling rate는 16,000으로 설정한다.
ds = Dataset.from_dict({"audio": [path for path in df["raw_data"]],
                       "transcripts": [transcript for transcript in df["transcript"]]}).cast_column("audio", Audio(sampling_rate=16000))

In [14]:
ds

Dataset({
    features: ['audio', 'transcripts'],
    num_rows: 10000
})

In [15]:
# 데이터셋을 훈련 데이터와 테스트 데이터, 검증 데이터로 분할
train_testvalid = ds.train_test_split(test_size=0.2)
test_valid = train_testvalid["test"].train_test_split(test_size=0.5)
datasets = DatasetDict({
    "train": train_testvalid["train"],
    "test": test_valid["test"],
    "valid": test_valid["train"]})

In [16]:
datasets

DatasetDict({
    train: Dataset({
        features: ['audio', 'transcripts'],
        num_rows: 8000
    })
    test: Dataset({
        features: ['audio', 'transcripts'],
        num_rows: 1000
    })
    valid: Dataset({
        features: ['audio', 'transcripts'],
        num_rows: 1000
    })
})

In [17]:
# 작성한 데이터셋을 로컬 디스크에 저장
print(sys.platform)
dest_folder = os.getcwd()
datasets.save_to_disk(os.path.join(dest_folder, 'sbs_datasets'))

win32


Saving the dataset (0/3 shards):   0%|          | 0/8000 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/1000 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/1000 [00:00<?, ? examples/s]

### Dataset Preprocessing

In [18]:
import os
import sys
from datasets import load_from_disk

In [19]:
# 작성한 데이터셋을 로컬 디스크로 부터 읽어 온다.
dest_folder = os.getcwd()
sbs_datasets  = load_from_disk(os.path.join(dest_folder, 'sbs_datasets'))

데이터셋에 대하여 다음의 작업을 수행할 함수를 선언한다.
1. 오디오 데이터를 로드하고 리샘플링을 실시
2. feature extractor를 통해 1차원 오디오 배열을 log-Mel spectrogram으로 변환
3. tokenizer를 이용해 전사 데이터를 label ids로 변환

In [20]:
def prepare_dataset(batch):
    # 오디오 파일을 16kHz로 로드
    audio = batch["audio"]

    if 'base' in whisper_model_name:
        feature_extractor.feature_size = 80
    else:
        feature_extractor.feature_size = 128

    # input audio array로부터 log-Mel spectrogram 변환
    batch["input_features"] = feature_extractor(audio["array"], sampling_rate=audio["sampling_rate"]).input_features[0]

    # target text를 label ids로 변환
    batch["labels"] = tokenizer(batch["transcripts"]).input_ids
    return batch

In [21]:
# 데이터 전처리 함수를 데이터셋 전체에 적용
sbs_datasets = datasets.map(prepare_dataset, remove_columns=datasets.column_names["train"], num_proc=None)

Map:   0%|          | 0/8000 [00:00<?, ? examples/s]

Map:   0%|          | 0/1000 [00:00<?, ? examples/s]

Map:   0%|          | 0/1000 [00:00<?, ? examples/s]

In [22]:
# 전처리 작업이 오래 걸릴 수 있으므로, 전처리가 완료된 데이터셋을 Hub에 저장하는 것을 추천한다.
sbs_datasets.save_to_disk(os.path.join(dest_folder, 'sbs_datasets_processed'))

Saving the dataset (0/16 shards):   0%|          | 0/8000 [00:00<?, ? examples/s]

Saving the dataset (0/2 shards):   0%|          | 0/1000 [00:00<?, ? examples/s]

Saving the dataset (0/2 shards):   0%|          | 0/1000 [00:00<?, ? examples/s]

### Download preprocessed dataset

In [23]:
import os
import sys
import datasets

dest_folder = os.getcwd()

# Hub로부터 전처리가 완료된 데이터셋을 로드
print(os.path.join(dest_folder, 'sbs_datasets_processed'))
from datasets import load_from_disk
sbs_datasets_processed = load_from_disk(os.path.join(dest_folder, 'sbs_datasets_processed'))
sbs_datasets_processed

c:\Users\agate\Documents\mirae_whisper\sbs_datasets_processed


DatasetDict({
    train: Dataset({
        features: ['input_features', 'labels'],
        num_rows: 8000
    })
    test: Dataset({
        features: ['input_features', 'labels'],
        num_rows: 1000
    })
    valid: Dataset({
        features: ['input_features', 'labels'],
        num_rows: 1000
    })
})

### Prepare Training

이제 데이터가 준비되었으므로 training pipeline을 수행할 준비가 되었다. Hugging face의 'Trainer'를 사용하면 다음의 단계에 따라 간단히 수행할 수 있다.

1. data collator 선언 : data collator는 우리가 전처리한 데이터를 받아 모델에 입력할 수 있는 PyTorch Tensor 형태로 변환해준다.
2. 평가 지표 : 모델을 평가하기 위한 지표를 설정한다. 영어의 경우 Word Error Rate(WER)를 주로 사용하지만, 교착어인 한국어는 CER(Charactor Error Rate)를 사용하는 편이 더 효과적이다.
3. pre-trained model checkpoint 로드 : pre-trained model checkpoint를 로드하고 학습을 위한 설정을 진행한다.
4. Training argument의 정의 : Trainer가 활용할 수 있도록 argument를 설정한다.


#### Define a Data Collator

Sequence-to-sequence 발화 모델을 위한 Data collator는 input_feature와 label을 독립적으로 다룬다는 점에서 독특한 성격을 보인다. input_feature는 feature extractor로, label은 tokenizer로 다루어야 한다.

input_feature는 30초 길이로 패딩되고 고정된 차원의 log_mel spectrogram으로 변환되었으므로, 우리가 할 일은 이를 PyTorch tensor로 변환하는 것뿐이다. 이를 위해 .pad 메서드의 return_tensor=pt 인자를 사용한다. 이때 이미 패딩이 완료되었으므로 여기서 추가적인 패딩 작업이 이루어지지는 않으며, 그저 input_feature를 PyTorch tensor로 변환하기만 할 것이다.

반면, label은 아직 패딩 작업이 이루어지지 않았다. 따라서 먼저 tokenizer의 .pad 메서드를 이용해 패딩 작업을 실시할 것이다. 패딩 토큰들은 -100으로 치환되며, 따라서 이 토큰들은 모델이 loss를 계산할 때는 이용되지 않을 것이다. 그리고 이후 training 작업 동안 우리는 label sequence의 시작 부분에 있는 transcript 토큰을 잘라낼 것이다.

앞서 선언했던 WhisperPrecessor를 이용해 feature extractor와 tokenizer 모두를 사용할 수 있다.

In [24]:
import torch
from dataclasses import dataclass
from typing import Any, Dict, List, Union

In [25]:
@dataclass
class DataCollatorSpeechSeq2SeqWithPadding:
    processor: Any

    def __call__(self, features: List[Dict[str, Union[List[int], torch.Tensor]]]) -> Dict[str, torch.Tensor]:
        # 인풋 데이터와 라벨 데이터의 길이가 다르며, 따라서 서로 다른 패딩 방법이 적용되어야 한다. 그러므로 두 데이터를 분리해야 한다.
        # 먼저 오디오 인풋 데이터를 간단히 토치 텐서로 반환하는 작업을 수행한다.
        input_features = [{"input_features": feature["input_features"]} for feature in features]
        batch = self.processor.feature_extractor.pad(input_features, return_tensors="pt")

        # Tokenize된 레이블 시퀀스를 가져온다.
        label_features = [{"input_ids": feature["labels"]} for feature in features]
        # 레이블 시퀀스에 대해 최대 길이만큼 패딩 작업을 실시한다.
        labels_batch = self.processor.tokenizer.pad(label_features, return_tensors="pt")

        # 패딩 토큰을 -100으로 치환하여 loss 계산 과정에서 무시되도록 한다.
        labels = labels_batch["input_ids"].masked_fill(labels_batch.attention_mask.ne(1), -100)

        # 이전 토크나이즈 과정에서 bos 토큰이 추가되었다면 bos 토큰을 잘라낸다.
        # 해당 토큰은 이후 언제든 추가할 수 있다.
        if (labels[:, 0] == self.processor.tokenizer.bos_token_id).all().cpu().item():
            labels = labels[:, 1:]

        batch["labels"] = labels

        return batch


In [56]:
# 훈련시킬 모델의 processor, tokenizer, feature extractor 로드
from transformers import WhisperTokenizer,  WhisperFeatureExtractor
from transformers import WhisperProcessor

processor = WhisperProcessor.from_pretrained(whisper_model_name, language="Korean", task="transcribe")
tokenizer = WhisperTokenizer.from_pretrained(whisper_model_name, language="Korean", task="transcribe")
feature_extractor = WhisperFeatureExtractor.from_pretrained(whisper_model_name)

In [57]:
# 데이터 콜레이터 초기화
data_collator = DataCollatorSpeechSeq2SeqWithPadding(processor=processor)

#### Evaluation Metrics

검증 데이터셋에 사용할 evaluation metrics를 정의한다. 영어의 경우 WER을 사용하지만, 한국어 데이터이므로 CER을 사용하는 것이 더 적절할 것이다.

In [58]:
import evaluate
metric = evaluate.load('cer')

이제 모델의 예측에 대해 CER metric을 반환할 함수를 선언한다.

compute_metrics라는 이름의 이 함수는 가장 먼저 label_ids에 있는 -100을 pad_token으로 치환한다(data collator가 비용함수를 계산할 때 pad token을 무시하도록 하기 위해 설정한). 그 뒤 예측치와 label ids를 문자열로 변환하고 예측치와 정답을 비교하여 CER을 계산해낸다.

In [59]:
def compute_metrics(pred):
    pred_ids = pred.predictions
    label_ids = pred.label_ids

    # pad_token을 -100으로 치환
    label_ids[label_ids == -100] = tokenizer.pad_token_id

    # metrics 계산 시 special token들을 빼고 계산하도록 설정
    pred_str = tokenizer.batch_decode(pred_ids, skip_special_tokens=True)
    label_str = tokenizer.batch_decode(label_ids, skip_special_tokens=True)

    cer = 100 * metric.compute(predictions=pred_str, references=label_str)

    return {"cer": cer}

### Load a Pre-Trained Checkpoint

pre-trained Whisper 모델의 checkpoint를 로드한다.

In [60]:
from transformers import WhisperForConditionalGeneration

model = WhisperForConditionalGeneration.from_pretrained(whisper_model_name)

Whisper 모델은 문장의 자동 생성이 시작되기 전 모델의 출력으로 강제되는 토큰(forced_decoder_ids)이 있다. 이 token ids는 전사 언어와 zero-shot ASR 작업에 영향을 미친다. 파인 튜닝을 위해 우리는 이 ids를 None으로 바꿔주어야 한다. 우리는 모델이 정확한 언어(한국어)로 예측하고 전사하도록 훈련할 것이기 때문이다.

또한 문장의 생성 중 완전하게 억제되는 토큰들도 있다(suppress_tokens). 이 토큰들은 로그 확률을 -inf로 설정하며, 따라서 샘플링되지 않는다. 우리는 이 토큰들을 비어 있는 리스트로 치환할 것이다. 즉, 어떤 토큰도 억제되지 않는다.

In [61]:
model.config.forced_decoder_ids = None
model.config.suppress_tokens = []

### Define the Training Arguments

마지막 단계로서, 트레이닝을 위한 모든 파라미터들을 정의해야 한다. 각각의 파라미터들은 다음과 같은 의미를 갖는다.

- output_dir : 모델의 가중치를 저장하기 위한 경로를 설정한다. 이 경로는 허깅 페이스 허브의 리포지토리 이름으로도 설정 가능하다.
- generation_max_length : 평가 작업 동안 자기회귀적으로 생성되는 토큰들의 최대 길이를 설정한다.
- save_steps : 훈련 동안, 이 파라미터에 설정한 step마다 중간 체크포인트가 비동기적으로 저장 및 업로드될 것이다.
- eval_steps : 훈련 동안, 이 파라미터에 설정한 step마다 체크포인트에 대한 평가가 이루어질 것이다.
- report_to : 훈련 로그를 어디에 저장할지를 설정한다. 'azure_ml', 'comet_ml', 'mlflow', 'neptune', 'tensorboard' 그리고 'wandb'를 지원하며, 기본값은 'tensorboard'이다.

In [65]:
# import wandb
from transformers import Seq2SeqTrainingArguments

# os.environ["WANDB_PROJECT"] = "asr_whisper_large_v3_turbo"
# os.environ["WANDB_LOG_MODEL"] = "checkpoint"

log_dir = os.path.join(dest_folder, 'fine-tuned_models')

training_args = Seq2SeqTrainingArguments(
    output_dir=log_dir,  # 원하는 리포지토리 이름을 임력한다.
    per_device_train_batch_size=16,
    gradient_accumulation_steps=1,  # 배치 크기가 2배 감소할 때마다 2배씩 증가
    learning_rate=1e-5,
    warmup_steps=5,
    max_steps=15,  # epoch 대신 설정
    # gradient_checkpointing=True,
    fp16=True,
    eval_strategy="steps",
    per_device_eval_batch_size=8,
    predict_with_generate=True,
    generation_max_length=225,
    save_steps=5,
    eval_steps=5,
    logging_steps=5,
    report_to="tensorboard",
    # report_to="wandb"
    load_best_model_at_end=True,
    metric_for_best_model="cer",  # 한국어의 경우 'wer'보다는 'cer'이 더 적합할 것
    greater_is_better=False,
    push_to_hub=False,
)


트레이닝 파라미터들의 설정이 끝났다면 트레이너를 설정한다.

In [66]:
from transformers import Seq2SeqTrainer

trainer = Seq2SeqTrainer(
    args=training_args,
    model=model,
    train_dataset=sbs_datasets_processed["train"],
    eval_dataset=sbs_datasets_processed["valid"],  # or "test"
    data_collator=data_collator,
    compute_metrics=compute_metrics,
    processing_class=processor.feature_extractor
)


### Training

In [ ]:
os.environ['NCCL_DEBUG'] = 'WARN'
trainer.train()

c:\Users\agate\Documents\mirae_whisper\.venv\lib\site-packages\torch\utils\data\dataloader.py:665: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)


Step,Training Loss,Validation Loss,Cer
5,3.231100,2.910335,75.705349


c:\Users\agate\Documents\mirae_whisper\.venv\lib\site-packages\transformers\modeling_utils.py:3685: UserWarning: Moving the following attributes in the config to the generation config: {'max_length': 448, 'suppress_tokens': [], 'begin_suppress_tokens': [220, 50257]}. You are seeing this warning because you've set generation parameters in the model config, as opposed to in the generation config.
  if dtype_present_in_args and getattr(self, "quantization_method", None) == QuantizationMethod.QUARK:
c:\Users\agate\Documents\mirae_whisper\.venv\lib\site-packages\torch\utils\data\dataloader.py:665: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)


In [71]:
# Jupyter Notebook에서 TensorBoard 실행
%load_ext tensorboard
%tensorboard --logdir log_dir

Launching TensorBoard...

### Evaluation

#### Model Selection

In [1]:
# 파인 튜닝한 모델을 로드
from transformers import WhisperForConditionalGeneration, WhisperProcessor, WhisperFeatureExtractor, WhisperTokenizer

model = WhisperForConditionalGeneration.from_pretrained("/cul_m01/프리뷰노트/SBS_페쇄자막_데이터/fine-tuned_models/checkpoint-4000", local_files_only=True)

feature_extractor = WhisperFeatureExtractor.from_pretrained("/cul_m01/프리뷰노트/SBS_페쇄자막_데이터/fine-tuned_models/checkpoint-4000", local_files_only=True)
tokenizer = WhisperTokenizer.from_pretrained("/cul_m01/프리뷰노트/SBS_페쇄자막_데이터/fine-tuned_models/checkpoint-4000", local_files_only=True, language="Korean", task="transcribe")
processor = WhisperProcessor.from_pretrained("/cul_m01/프리뷰노트/SBS_페쇄자막_데이터/fine-tuned_models/checkpoint-4000", local_files_only=True)

OSError: Incorrect path_or_model_id: '/cul_m01/프리뷰노트/SBS_페쇄자막_데이터/fine-tuned_models/checkpoint-4000'. Please provide either the path to a local folder or the repo_id of a model on the Hub.

#### Training argument 설정

In [ ]:
from transformers import Seq2SeqTrainingArguments

training_args = Seq2SeqTrainingArguments(
    output_dir="repo_name",  # 원하는 리포지토리 이름을 임력한다.
    per_device_train_batch_size=16,
    gradient_accumulation_steps=1,  # 배치 크기가 2배 감소할 때마다 2배씩 증가
    learning_rate=1e-5,
    warmup_steps=500,
    max_steps=4000,
    gradient_checkpointing=True,
    fp16=True,
    evaluation_strategy="steps",
    per_device_eval_batch_size=8,
    predict_with_generate=True,
    generation_max_length=225,
    save_steps=1000,
    eval_steps=1000,
    logging_steps=25,
    report_to=["tensorboard"],
    load_best_model_at_end=True,
    metric_for_best_model="cer",  # 한국어의 경우 'wer'보다는 'cer'이 더 적합할 것
    greater_is_better=False,
    push_to_hub=False,
)


In [ ]:
from transformers import Seq2SeqTrainer

trainer = Seq2SeqTrainer(
    args=training_args,
    model=model,
    train_dataset=low_call_voices_prepreocessed["train"],
    eval_dataset=low_call_voices_prepreocessed["test"],  # for evaluation(not validation)
    data_collator=data_collator,
    compute_metrics=compute_metrics,
    tokenizer=processor.feature_extractor,
)


> Evaluation 진행

In [ ]:
trainer.evaluate()